# word2vec
이 장에서 다룰 것도 마찬가지로 단어의 분산표현이다. 여기서는 '추론에 기반한 방법'에 대해 알아볼 것이다.
먼저, word2vec을 실제로 코드로 짜보자!

## 추론 기반의 방법과 신경망
단어를 벡터로 표현한 방법 중, 성공한 것은 '카운트 기반 방법'과 '추론 기반 방법'이다. 이 두가지 방법의 배경에는 분배가설이 있다.

### 카운트 기반의 방법의 문제점
카운트 기반의 문제점은, 역시 어휘가 많아질 때 늘어난다. 어휘량이 100만개라고 할 때, 이 방식으로는 100만 x 100만의 거대한 행렬을 만들게 된다. 이 때, 이렇게 커다란 행렬에 대해 특이값분해를 하는 것은 현실적으로 불가능하다.

카운트 기반 방법은 코퍼스 전체의 통계 데이터(cocurrence matrix나 ppmi 등)을 이용해, 1회의 처리(SVD 등)으로 단어의 분산표현을 얻는다. 반면, 추론 기반 방법은 미니배치를 이용한 학습을 진행한다. 따라서 연산의 부하가 덜 걸리게 된다.

### 추론 기반의 방법 개요
추론 기반은 말 그대로 문맥이 주어졌을 때, 중간에 구멍을 뚫어놓고 어떤 단어가 출현할지 추측하는 것이다. 이런 추측을 거듭하여, 단어의 출현 패턴을 학습한다.

우리는 앞으로 신경망으로 단어를 처리할 것이다. 그러나, 'you'나 'will'같은 단어를 컴퓨터가 그대로 처리하지는 못한다. 신경망에 통과시키기 위해서는, 벡터로 변환할 필요가 있다. 이 방법 중 하나는 단어를 one-hot vector로 변환하는 것이다.

따라서, 모든 단어를 one-hot vector로 변환하면 이 벡터의 집합은 신경망을 구성하는 다양한 '레이어'에 의해 처리할 수 있다.
그렇다면 예를 들어보자. 

아래 예에서 c와 W의 내적은, 단순히 W의 행벡터를 "끄집어 내는" 것에 지나지 않는다. 비효율적이다. 그래서 ch4에서는 이를 개량한 코드를 작성할 것이다.

In [2]:
import numpy as np

# bias는 생략
c = np.array([[1, 0, 0, 0, 0, 0, 0]]) # 입력층. 단어ID에 해당하는 원소는 1이고, 나머지는 0인 ONE-HOT VECTOR.
W = np.random.randn(7,3) # weight
h = np.dot(c, W) # 중간 노드
print(h)

[[-0.32652697 -0.0091539   0.30483792]]


In [5]:
# Matmul layer를 사용해서 다시 쓸 수 있다.
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None

    def forward(self, x):
        W, = self.params
        out = np.dot(x, W)
        self.x = x
        return out

    def backward(self, dout):
        W, = self.params
        dx = np.dot(dout, W.T)
        dW = np.dout(self.x.T, dout)
        self.grads[0][...] = dW
        return dx

c = np.array([[1, 0, 0, 0, 0, 0, 0]]) # 입력층. 단어ID에 해당하는 원소는 1이고, 나머지는 0인 ONE-HOT VECTOR.
W = np.random.randn(7,3) # weight
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[-1.36072619 -1.54564841 -0.06030721]]


이제 신경망을 만들어 보자. word2vec에서 쓰이는 CBOW(Continuous Bag-of-words)라고 쓰이는 모델을 사용할 것이다.

사실 word2vec이라는 용어는 모델이 아니라, 프로그램의 이름이다. 정확히는 CBOW 모델과 skip-gram 모델이라는 두 가지 모델이 word2vec에 사용되는 신경망이다.

CBOW 모델은, 문맥에서 타겟(가운데 단어)을 추측하는 것을 목적으로 한 신경망이다. CBOW 모델이 입력받는 것은 문맥(컨텍스트)이다. 이 컨텍스트는 `['i', 'love', 'you']`와 같은 단어의 리스트로 표현된다. 이를 one-hot 표현으로 바꿔 CBOW 모델이 처리할 수 있게 변환할 것이다.

CBOW의 입력층은 문맥의 단어 개수에 따라 결정된다. 만약 문맥으로 N개의 단어를 다룬다면 입력층은 N개가 된다. 입력층에서 중간층의 변환은, 같은 Affine layer(weight: Win)에 의해 이루어진다. 그리고 중간층에서 출력층의 변환은 다른 Affine layer(weight: Wout)에 의해 이루어진다.